### Cats vs Dogs dataseti

In [ ]:
import tensorflow_datasets as tfds

In [ ]:
data_name = 'cats_vs_dogs'

In [ ]:
dataset, info = tfds.load(
    data_name,
    split=['train'],
    data_dir="Cats_and_Dogs\\",
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

In [ ]:
info

In [ ]:
fig = tfds.show_examples(dataset[0], info)

In [ ]:
dataset = dataset[0].take(13000)
images,labels = [],[]
for image, label in tfds.as_numpy(dataset):
    images.append(image)
    labels.append(label)

In [ ]:
X_train = images[:10000]
X_test = images[10000:]

Y_train = labels[:10000]
Y_test = labels[10000:]

In [ ]:
print(f"Train data tarkibidagi 0-class elementlari soni:{Y_train.count(0)}")
print(f"Train data tarkibidagi 1-class elementlari soni:{Y_train.count(1)}")
print(f"Test data tarkibidagi 0-class elementlari soni:{Y_test.count(0)}")
print(f"Test data tarkibidagi 1-class elementlari soni:{Y_test.count(1)}")

In [ ]:
import numpy as np
import cv2
import os

In [ ]:
def resize(data):
    for i in range(len(data)):
        data[i] = cv2.cvtColor(data[i],cv2.COLOR_BGR2RGB)
        data[i] = np.array(cv2.resize(data[i],(256,256),interpolation = cv2.INTER_AREA))
    return data

X_train = np.array(resize(X_train)).astype('float32')/255.0
X_test = np.array(resize(X_test)).astype('float32')/255.0

Y_train = np.array(Y_train).astype('uint8')
Y_test = np.array(Y_test).astype('uint8')

print(f"X_train shape:{X_train.shape}")
print(f"X_test shape:{X_test.shape}")
print(f"Y_train shape:{Y_train.shape}")
print(f"Y_test shape:{Y_test.shape}")

In [ ]:
from tensorflow.keras import Model
import tensorflow as tf

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((X_train, Y_train)).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, Y_test)).batch(32)

In [ ]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        # Convolution Layer
        self.block1_conv = tf.keras.layers.Conv2D(16, 3, activation=tf.keras.activations.relu, padding="same")
        self.block2_conv = tf.keras.layers.Conv2D(32, 3, activation=tf.keras.activations.relu, padding="same")
        self.block3_conv = tf.keras.layers.Conv2D(64, 3, activation=tf.keras.activations.relu, padding="same")
        self.block4_conv = tf.keras.layers.Conv2D(128, 3, activation=tf.keras.activations.relu, padding="same")
        
        self.pool = tf.keras.layers.MaxPooling2D((2,2))
        
        # Classification Layer
        self.flatten = tf.keras.layers.Flatten()
        
        self.hidden1 = tf.keras.layers.Dense(512, activation=tf.keras.activations.relu)
        self.hidden2 = tf.keras.layers.Dense(256)
        self.hidden3 = tf.keras.layers.Dense(10)
        
        self.output_layer = tf.keras.layers.Dense(2, activation=tf.keras.activations.softmax)
        
    def call(self, x):
        # Block 1
        
        conv1 = self.block1_conv(x) # 256x256x16
        pooling = self.pool(conv1) # 128x128x16
        
        # Block 2
        conv2 = self.block2_conv(pooling) # 128x128x32
        pooling = self.pool(conv2) # 64x64x32
        
        # Block 3
        conv3 = self.block3_conv(pooling) # 64x64x64
        pooling = self.pool(conv3) # 32x32x64
        
        # Block 3
        conv4 = self.block4_conv(pooling) # 32x32x128
        pooling = self.pool(conv4) # 16x16x128
        
        flatten_layer = self.flatten(pooling)
        
        dense1 = self.hidden1(flatten_layer)
        dense2 = self.hidden2(dense1)
        dense3 = self.hidden3(dense2)
        
        return self.output_layer(dense3)

model = MyModel()

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer, model=model)

latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)

if latest_checkpoint:
    checkpoint.restore(latest_checkpoint)
    print(f"Restored checkpoint from {latest_checkpoint}")

else:
    print("Checkpoint not found. Training from scratch.")

In [ ]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [ ]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:

        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
        
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)
    
    checkpoint.write(file_prefix=checkpoint_dir+"/ckpt")

In [ ]:
@tf.function
def test_step(images, labels):
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [ ]:
EPOCHS = 5

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    print(
    f'** {epoch + 1}-epoch:\n '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}\n'
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}')

In [ ]:
model.summary()

In [ ]:
model.save('Model')

### Modelni sinovdan o'tkazamiz 

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('Model') 

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

#### Dog sinfi uchun misol

In [ ]:
image_dog = cv2.imread("Cats_and_Dogs\examples\cats_vs_dogs (102).jpg")
plt.imshow(image_dog)

In [ ]:
image_dog = np.array(cv2.resize(image_dog,(256,256),interpolation = cv2.INTER_AREA))
image_dog = np.array(image_dog).astype('float32')/255.0

In [ ]:
image_dog.shape

In [ ]:
image = image_dog.reshape(1,256,256,3)

In [ ]:
one_hot_encoding_label = model.predict(image)

In [ ]:
one_hot_encoding_label

In [ ]:
if np.argmax(one_hot_encoding_label)==1:
    print("Dog")
    
else:
    print("Cat")

#### Cat sinfi uchun misol

In [ ]:
image_cat = cv2.imread("Cats_and_Dogs\examples\cats_vs_dogs (97).jpg")
plt.imshow(image_cat)

In [ ]:
image_cat = np.array(cv2.resize(image_cat,(256,256),interpolation = cv2.INTER_AREA))
image_cat = np.array(image_cat).astype('float32')/255.0

In [ ]:
image_cat.shape

In [ ]:
image = image_cat.reshape(1,256,256,3)

In [ ]:
one_hot_encoding_label = model.predict(image)

In [ ]:
one_hot_encoding_label

In [ ]:
if np.argmax(one_hot_encoding_label)==1:
    print("Dog")
    
else:
    print("Cat")